<a href="https://colab.research.google.com/github/seonmi0422/practice/blob/main/gradio_appcode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Group quiz: Task #1 (App deployment) & Share the link on Padlet

## 🚩 Instruction for deployment on Huggingface. When deploying, change the code as described below:

1. First run the code below to check whether it is working.
2. Make a hyperlink of your story (the story assigned to your group) text below the title "Application". Currently the link is empty (File not found).
=> Use the link "https://github.com/MK316/Digital-Literacy-Class/blob/main/materials/storytelling.md"

3. [Extra point] Provide a brief description of how the application works for new visitors on your application (learners for example)

3. Change the title:

+ From "Application" to "Miran's Sentence Reading Application (Group 0)"
=> Change the name to yours.
+ Change the icon 🗣️ to something else :-)

4. Change the button label:

+ From "Split into Sentences' to "Show sentence list"

5. When you successfully deploy the app on huggingface, share your link on Padlet. (All members should succeed and post one)
Note: The link name should be the app link, not hugggingface address. (Short form of the application link; introduced in the lecture video)



---
## Code to use

In [ ]:
%%capture
!pip install gradio gtts

In [ ]:
#@markdown App code
import gradio as gr
from gtts import gTTS
import pandas as pd
import re
from io import BytesIO

def split_text_to_sentences(text):
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a.z])(?<=\.|\?)\s', text)
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
    df = pd.DataFrame({'Index': range(1, len(sentences)+1), 'Sentences': sentences})
    return df

def generate_audio(sentence_number, df):
    try:
        sentence_number = int(sentence_number)
        if sentence_number in df['Index'].values:
            selected_sentence = df.loc[df['Index'] == sentence_number, 'Sentences'].values[0]
            tts = gTTS(text=selected_sentence, lang='en')
            audio_fp = BytesIO()
            tts.write_to_fp(audio_fp)
            audio_fp.seek(0)
            return audio_fp.getvalue(), selected_sentence  # 👈 Return the sentence itself
        else:
            return None, "⚠️ Invalid sentence index."
    except ValueError:
        return None, "⚠️ Please enter a number."

with gr.Blocks() as app:
    gr.Markdown("#  ❤️Seonmi's Sentence Reading Application(Group3)")
    gr.Markdown("function 1:separate all the sentences written within a paragraph and assigns them order-based numbers,function 2: of the app will convert the selected sentence into audio")
    gr.Markdown("🍋[Story sample](https://github.com/thelighterside/Practice/blob/main/Story03.md)")
    text_input = gr.Textbox(label="Enter Text", lines=4, placeholder="Paste or type your text here")
    process_button = gr.Button("Split into Sentences")
    sentences_output = gr.Dataframe()
    sentence_number_input = gr.Textbox(label="Enter Sentence Index", placeholder="e.g., 1")
    generate_audio_button = gr.Button("Generate Audio")

    message_output = gr.Label(label="Label")
    audio_output = gr.Audio()

    def process_text(text):
        return split_text_to_sentences(text)


    def handle_audio_generation(sentence_number, df):
        audio, message = generate_audio(sentence_number, df)
        return audio, message

    process_button.click(
        fn=process_text,
        inputs=text_input,
        outputs=sentences_output
    )

    generate_audio_button.click(
        fn=handle_audio_generation,
        inputs=[sentence_number_input, sentences_output],
        outputs=[audio_output, message_output]
    )

app.launch()
